In [ ]:
pip install protobuf==3.19.6

In [ ]:
pip install --upgrade protobuf

In [ ]:
import subprocess
import os
import signal
import re

def get_gpu_processes():
    result = subprocess.run(['nvidia-smi', '--query-compute-apps=pid', '--format=csv,noheader'], stdout=subprocess.PIPE)
    output = result.stdout.decode('utf-8')
    pids = [int(pid) for pid in output.strip().split('\n') if pid]
    return pids

def kill_processes(pids):
    for pid in pids:
        try:
            os.kill(pid, signal.SIGKILL)
            print(f"Process {pid} has been killed.")
        except Exception as e:
            print(f"Could not kill process {pid}: {e}")

if __name__ == "__main__":
    gpu_pids = get_gpu_processes()
    if gpu_pids:
        print(f"Found processes running on GPU: {gpu_pids}")
        kill_processes(gpu_pids)
    else:
        print("No GPU processes found.")


In [ ]:
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image
import cv2
import os

strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

def load_and_resize_image(path, new_width=1024, new_height=1024):
    """Load an image from file and resize it."""
    try:
        img = Image.open(path).convert('RGB').resize((new_width, new_height), Image.ANTIALIAS)
        return np.array(img)
    except Exception as e:
        print(f"Failed to load or resize image {path}: {str(e)}")
        return None

base_model_dir = '/scratch/rk42218/Building_Detection_scratch/exported_models_new/'
label_map_path = '/home/rk42218/DATA_SET_1024/data_split/label_map.pbtxt'
image_dir = '/home/rk42218/DATA_SET_1024/data_split/test/images/'
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

print(f"Number of models found: {len(os.listdir(base_model_dir))}")
for model_folder in os.listdir(base_model_dir):
    if model_folder == '.ipynb_checkpoints':
        continue
    print(f"Processing model folder: {model_folder}")

    model_dir = os.path.join(base_model_dir, model_folder, 'saved_model')
    model_pb_path = os.path.join(model_dir, 'saved_model.pb')
    print(f"Checking model path: {model_pb_path}")

    if not os.path.exists(model_pb_path):
        print(f"SavedModel not found in {model_dir}. Skipping...")
        continue

    output_dir = os.path.join('/scratch/rk42218/Building_Detection_scratch/exported_models_new/new_created_images_new_05', model_folder)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory at {output_dir}")

    with strategy.scope():
        model = tf.saved_model.load(model_dir)

    detected_building_images = []
    image_files = os.listdir(image_dir)
    print(f"Number of images to process: {len(image_files)}")
    for image_name in image_files:
        image_path = os.path.join(image_dir, image_name)
        print(f"Processing image {image_path}")
        image_np = load_and_resize_image(image_path)
        if image_np is None:
            continue

        try:
            input_tensor = tf.convert_to_tensor([image_np], dtype=tf.uint8)

            # Run detection
            detections = model(input_tensor)

            # Apply visualization
            viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np,
                detections['detection_boxes'][0].numpy(),
                detections['detection_classes'][0].numpy().astype(int),
                detections['detection_scores'][0].numpy(),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8,
                min_score_thresh=0.5)  # Threshold for visualization

            # Save images only if buildings are detected
            if np.any(detections['detection_scores'][0].numpy() > 0.5):
                for i, score in enumerate(detections['detection_scores'][0].numpy()):
                    if score > 0.5 and category_index[detections['detection_classes'][0][i].numpy()]['name'] == 'building':
                        output_image_path = os.path.join(output_dir, image_name)
                        cv2.imwrite(output_image_path, cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))
                        detected_building_images.append(image_name)
                        print(f"Building detected in {image_name}, saved to {output_image_path}")
                        break

        except Exception as e:
            print(f"Error processing {image_name}: {str(e)}")

    if detected_building_images:
        print(f"Images with detected buildings for model {model_folder}: {detected_building_images}")
    else:
        print(f"No buildings detected for model {model_folder}")
